In [1]:
!pip install polars numpy pandas lightgbm optuna scikit-learn pyarrow -q
from datetime import datetime
import os
import polars as pl
import numpy as np
import pandas as pd
import yaml
import random
import lightgbm as lgb
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score
import gc

import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="X does not have valid feature names")

In [2]:
# -------- Fijar semillas --------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

## Generacion de la clase_ternaria

In [3]:
df = pl.read_csv(
    "datasets/competencia_01_crudo.csv",
    infer_schema_length=100000
)
df.head()

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,tcuentas,ccuenta_corriente,mcuenta_corriente_adicional,mcuenta_corriente,ccaja_ahorro,mcaja_ahorro,mcaja_ahorro_adicional,mcaja_ahorro_dolares,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_personales,mprestamos_personales,cprestamos_prendarios,mprestamos_prendarios,cprestamos_hipotecarios,…,Master_msaldodolares,Master_mconsumospesos,Master_mconsumosdolares,Master_mlimitecompra,Master_madelantopesos,Master_madelantodolares,Master_fultimo_cierre,Master_mpagado,Master_mpagospesos,Master_mpagosdolares,Master_fechaalta,Master_mconsumototal,Master_cconsumos,Master_cadelantosefectivo,Master_mpagominimo,Visa_delinquency,Visa_status,Visa_mfinanciacion_limite,Visa_Fvencimiento,Visa_Finiciomora,Visa_msaldototal,Visa_msaldopesos,Visa_msaldodolares,Visa_mconsumospesos,Visa_mconsumosdolares,Visa_mlimitecompra,Visa_madelantopesos,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i64,i64,i64,f64,f64,i64,f64,f64,f64,i64,f64,i64,i64,f64,i64,i64,f64,i64,i64,f64,i64,f64,i64,f64,i64,…,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,i64,f64,i64,i64,f64,i64,i64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,i64,f64,i64,i64,f64
249221323,202101,1,0,0,46,93,3070.24,15691.1,1278.11,890.96,636.86,8,1,1,0.0,-314.48,2,25761.53,0.0,10240.09,1,22890.77,1,5,35013.67,1,10,15929.92,1,13,27145.02,0,0.0,0,0.0,0,…,5171.55,10853.35,58.6,70380.0,0.0,0.0,4,0.0,-10385.48,0.0,2814,10853.35,7,0,9290.16,0,0,63342.0,-849,null,43251.02,50733.44,0.0,6743.38,1.36,70380.0,0.0,0.0,4,0.0,-20833.47,0.0,2814,6743.38,6,0,9559.95
249227600,202101,1,0,0,42,278,496.21,15321.8,418.97,-1138.55,1204.4,9,1,1,0.0,0.0,2,84736.17,0.0,30617.86,1,157762.37,1,19,20489.02,1,11,13110.17,1,0,0.0,1,85831.92,0,0.0,0,…,0.0,null,null,84456.0,null,null,4,0.0,null,null,3524,null,null,null,0.0,0,0,152020.8,-1063,null,11310.74,13267.49,0.0,9419.9,0.0,168912.0,0.0,0.0,11,13267.49,-9662.51,0.0,5541,9419.9,8,0,1900.26
249234235,202101,1,0,0,54,17,806.56,10549.8,174.26,546.73,-7.68,7,1,1,0.0,-129.29,2,0.0,0.0,0.0,1,-157.45,1,0,0.0,1,6,6156.93,1,0,0.0,0,0.0,0,0.0,0,…,0.0,null,null,225216.0,null,null,11,0.0,null,null,243,null,null,null,0.0,0,0,202694.4,-1702,null,19029.85,22322.01,0.0,2821.32,0.0,225216.0,0.0,0.0,4,0.0,-4692.0,0.0,577,2821.32,2,0,6885.51
249244449,202101,1,0,0,61,290,27485.71,65621.74,23590.68,-317.72,3638.29,7,1,1,0.0,0.0,2,252326.77,0.0,30200.06,1,271890.17,6,0,0.0,1,8,23509.71,1,0,0.0,0,0.0,0,0.0,0,…,0.0,null,null,84456.0,null,null,18,0.0,null,null,6752,null,null,null,0.0,0,0,76010.4,-879,null,20282.89,23791.82,0.0,23509.71,0.0,84456.0,0.0,0.0,25,23800.17,-20349.22,0.0,7916,23509.71,8,0,1219.92
249244739,202101,1,0,0,66,320,3306.82,48787.99,3274.95,-2002.76,2029.93,8,1,1,0.0,-1141.96,3,48569.5,20085.44,20635.82,1,43125.7,2,8,8094.87,1,15,38271.01,1,2,1613.88,0,0.0,0,0.0,0,…,627.0,619.84,7.1,197064.0,0.0,0.0,4,0.0,-61284.59,0.0,1958,619.84,1,0,985.32,0,0,177357.6,-1337,null,32883.64,38572.5,0.0,20397.54,0.0,197064.0,0.0,0.0,4,0.0,-88178.37,0.0,1958,20397.54,8,0,2627.52


In [4]:
def agregar_clase_ternaria(df: pl.DataFrame) -> pl.DataFrame:
    """
    Agrega columna clase_ternaria:
    - periodo_ultimo -> clase_ternaria = None
    - periodo_anteultimo -> solo puede ser BAJA+1 o None
    - resto -> CONTINUA / BAJA+1 / BAJA+2
    """
    # calculo periodo0
    df = df.with_columns(
        ((pl.col("foto_mes") // 100) * 12 + (pl.col("foto_mes") % 100)).alias("periodo0")
    )

    # calculo topes globales
    periodo_ultimo = df["periodo0"].max()
    periodo_anteultimo = periodo_ultimo - 1

    # ordeno y calculo leads (shift tipo "lead")
    df = (
        df.lazy()
        .sort(["numero_de_cliente", "periodo0"])
        .with_columns([
            pl.col("periodo0").shift(-1).over("numero_de_cliente").alias("periodo1"),
            pl.col("periodo0").shift(-2).over("numero_de_cliente").alias("periodo2"),
        ])
        # asigno clase_ternaria
        .with_columns([
            pl.when(
                # BAJA+1
                (pl.col("periodo0") < periodo_ultimo)
                & (pl.col("periodo1").is_null() | (pl.col("periodo0") + 1 < pl.col("periodo1")))
            )
            .then(pl.lit("BAJA+1"))
            .when(
                # BAJA+2
                (pl.col("periodo0") < periodo_anteultimo)
                & (pl.col("periodo0") + 1 == pl.col("periodo1"))
                & (pl.col("periodo2").is_null() | (pl.col("periodo0") + 2 < pl.col("periodo2")))
            )
            .then(pl.lit("BAJA+2"))
            .when(
                # CONTINUA: solo válido si no estamos en ultimo o anteultimo
                (pl.col("periodo0") < periodo_anteultimo)
            )
            .then(pl.lit("CONTINUA"))
            .otherwise(None)   # último y anteúltimo pueden quedar en None
            .alias("clase_ternaria")
        ])
        .collect()
    )

    return df


# Agregar la columna clase_ternaria al dataframe
df = agregar_clase_ternaria(df)
df = df.drop(["periodo0", "periodo1", "periodo2"])
df.head()

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,tcuentas,ccuenta_corriente,mcuenta_corriente_adicional,mcuenta_corriente,ccaja_ahorro,mcaja_ahorro,mcaja_ahorro_adicional,mcaja_ahorro_dolares,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_personales,mprestamos_personales,cprestamos_prendarios,mprestamos_prendarios,cprestamos_hipotecarios,…,Master_mconsumospesos,Master_mconsumosdolares,Master_mlimitecompra,Master_madelantopesos,Master_madelantodolares,Master_fultimo_cierre,Master_mpagado,Master_mpagospesos,Master_mpagosdolares,Master_fechaalta,Master_mconsumototal,Master_cconsumos,Master_cadelantosefectivo,Master_mpagominimo,Visa_delinquency,Visa_status,Visa_mfinanciacion_limite,Visa_Fvencimiento,Visa_Finiciomora,Visa_msaldototal,Visa_msaldopesos,Visa_msaldodolares,Visa_mconsumospesos,Visa_mconsumosdolares,Visa_mlimitecompra,Visa_madelantopesos,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i64,i64,i64,f64,f64,i64,f64,f64,f64,i64,f64,i64,i64,f64,i64,i64,f64,i64,i64,f64,i64,f64,i64,f64,i64,…,f64,f64,f64,f64,f64,i64,f64,f64,f64,i64,f64,i64,i64,f64,i64,i64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,i64,f64,i64,i64,f64,str
249221323,202101,1,0,0,46,93,3070.24,15691.1,1278.11,890.96,636.86,8,1,1,0.0,-314.48,2,25761.53,0.0,10240.09,1,22890.77,1,5,35013.67,1,10,15929.92,1,13,27145.02,0,0.0,0,0.0,0,…,10853.35,58.6,70380.0,0.0,0.0,4,0.0,-10385.48,0.0,2814,10853.35,7,0,9290.16,0,0,63342.0,-849,null,43251.02,50733.44,0.0,6743.38,1.36,70380.0,0.0,0.0,4,0.0,-20833.47,0.0,2814,6743.38,6,0,9559.95,"""CONTINUA"""
249221323,202102,1,0,0,46,94,3385.85,19103.66,1230.47,1631.3,206.19,8,1,1,0.0,0.0,2,8116.43,0.0,10536.47,1,10629.59,1,1,2315.5,1,11,17285.39,1,22,25713.45,0,0.0,0,0.0,0,…,10910.41,56.8,70380.0,0.0,0.0,-3,42765.48,-42426.83,59.3,2842,10910.41,17,0,8539.44,0,0,63342.0,-821,null,19687.47,23093.4,0.0,7715.26,1.77,70380.0,0.0,0.0,-3,50733.44,-50733.44,0.0,2842,7715.26,8,0,7413.36,"""CONTINUA"""
249221323,202103,1,0,0,46,95,5341.18,24020.21,755.85,2768.95,1140.11,8,1,1,0.0,-3933.18,2,4015.77,0.0,10789.84,1,4650.31,1,0,0.0,1,10,18976.09,1,16,45821.57,0,0.0,0,0.0,0,…,30992.11,39.83,70380.0,0.0,0.0,1,0.0,-42534.65,57.48,2873,30992.11,11,0,0.0,0,0,63342.0,-790,null,-2453.06,-2877.43,0.0,10703.0,1.53,70380.0,0.0,0.0,1,0.0,-50733.44,0.0,2873,10703.0,7,0,0.0,"""CONTINUA"""
249221323,202104,1,0,0,46,96,5356.62,29189.81,1470.2,2627.7,685.53,8,1,1,0.0,-1960.57,2,2136.11,0.0,10974.0,1,9358.46,1,0,0.0,1,10,19201.77,1,18,17373.08,0,0.0,0,0.0,0,…,10360.48,49.19,70380.0,0.0,0.0,2,0.0,-42579.46,40.31,2903,10360.48,14,0,4821.03,0,0,63342.0,-760,null,14545.66,17062.05,0.0,8094.73,1.51,70380.0,0.0,0.0,2,0.0,0.0,0.0,2903,8094.73,6,0,0.0,"""CONTINUA"""
249221323,202105,1,0,0,46,97,2489.25,31505.78,1008.8,952.16,309.95,8,1,1,0.0,-263.5,2,6204.56,0.0,11106.55,1,13240.63,1,1,2216.97,1,9,19278.3,1,14,16708.69,0,0.0,0,0.0,0,…,7293.92,43.34,70380.0,0.0,0.0,5,0.0,-21115.45,49.78,2934,7293.92,10,0,4586.43,0,0,63342.0,-729,null,17064.74,20016.94,0.0,9051.98,1.48,70380.0,0.0,0.0,5,0.0,-17062.05,0.0,2934,9051.98,6,0,2111.4,null


In [5]:
tabla = (
    df.pivot(
        values="numero_de_cliente",
        index="foto_mes",
        on="clase_ternaria",
        aggregate_function="len"
    )
    .fill_null(0)  # reemplazar nulls por 0
)

print(tabla)

shape: (6, 5)
┌──────────┬──────────┬────────┬────────┬────────┐
│ foto_mes ┆ CONTINUA ┆ null   ┆ BAJA+2 ┆ BAJA+1 │
│ ---      ┆ ---      ┆ ---    ┆ ---    ┆ ---    │
│ i64      ┆ u32      ┆ u32    ┆ u32    ┆ u32    │
╞══════════╪══════════╪════════╪════════╪════════╡
│ 202101   ┆ 160080   ┆ 0      ┆ 825    ┆ 622    │
│ 202102   ┆ 160292   ┆ 0      ┆ 1032   ┆ 831    │
│ 202103   ┆ 161119   ┆ 0      ┆ 951    ┆ 1039   │
│ 202104   ┆ 161333   ┆ 0      ┆ 1130   ┆ 955    │
│ 202105   ┆ 0        ┆ 162783 ┆ 0      ┆ 1134   │
│ 202106   ┆ 0        ┆ 164313 ┆ 0      ┆ 0      │
└──────────┴──────────┴────────┴────────┴────────┘


In [6]:
df.write_csv("datasets/competencia_01.csv.gz")

## Optimizacion Hiperparámetros

aqui debe cargar SU semilla primigenia
<br>recuerde cambiar el numero de experimento en cada corrida nueva

In [7]:
PARAM = {}
PARAM["experimento"] = 49422
PARAM["semilla_primigenia"] = 100129

In [8]:
# training and future
PARAM["train"] = [202101,202102,202103]
PARAM["train_final"] = [202101,202102,202103]
PARAM["future"] = [202104]
PARAM["kaggle"] = [202106]
PARAM["semilla_kaggle"] = 314159
PARAM["cortes"] = list(range(6000, 19001, 500))

In [9]:
# un undersampling de 0.1  toma solo el 10% de los CONTINUA
# undersampling de 1.0  implica tomar TODOS los datos

PARAM["trainingstrategy"] = {}
PARAM["trainingstrategy"]["undersampling"] = 1.0

### Definicion de Parametros

In [10]:
# Parámetros de LightGBM

PARAM["hyperparametertuning"] = {}
PARAM["hyperparametertuning"]["xval_folds"] = 5

# Parámetros fijos de LightGBM que serán sobreescritos por la parte variable de la BO
PARAM["lgbm"] = {}
PARAM["lgbm"]["param_fijos"] = {
    "boosting": "gbdt",  # puede ir 'dart', no probar 'random_forest'
    "objective": "binary",
    "metric": "auc",
    "first_metric_only": False,
    "boost_from_average": True,
    "feature_pre_filter": False,
    "force_row_wise": True,  # para reducir warnings
    "verbosity": -100,

    "seed": PARAM["semilla_primigenia"],

    "min_gain_to_split": 0,  # min_gain_to_split >= 0
    "min_sum_hessian_in_leaf": 0.001,  # min_sum_hessian_in_leaf >= 0.0
    "max_bin": 31,  # lo debo dejar fijo, no participa de la BO

    "pos_bagging_fraction": 1.0,  # 0.0 < pos_bagging_fraction <= 1.0
    "neg_bagging_fraction": 1.0,  # 0.0 < neg_bagging_fraction <= 1.0
    "is_unbalance": False,
    "scale_pos_weight": 1.0,

    "max_drop": 50,  # <=0 significa sin límite
    "skip_drop": 0.5,  # 0.0 <= skip_drop <= 1.0

    "extra_trees": False,

    "num_iterations": 1200
}

Aqui se definen los hiperparámetros de LightGBM que participan de la Bayesian Optimization

In [11]:
# Aquí se definen los límites de los hiperparámetros para la Optimización Bayesiana
PARAM["hyperparametertuning"]["hs"] = {
    "learning_rate":    {"type": "float", "bounds": (0.005, 0.05)},
    "feature_fraction": {"type": "float", "bounds": (0.3, 0.8)},
    "min_data_in_leaf": {"type": "int",   "bounds": (100, 30000)},
    "num_leaves":       {"type": "int",   "bounds": (10, 25000)},
    "max_depth":        {"type": "int",   "bounds": (-1, 30)},
    "lambda_l1":        {"type": "float", "bounds": (0.0, 10.0)},
    "lambda_l2":        {"type": "float", "bounds": (0.0, 10.0)},
    "bagging_fraction": {"type": "float", "bounds": (0.5, 1.0)}
}

A mayor cantidad de hiperparámetros, se debe aumentar las iteraciones de la Bayesian Optimization
<br> 30 es un valor muy tacaño, pero corre rápido
<br> deberia partir de 50, alcanzando los 100 si se dispone de tiempo

In [12]:
# Definir la cantidad de iteraciones para la optimización bayesiana
PARAM["hyperparametertuning"]["iteraciones"] = 50  # iteraciones bayesianas

In [13]:
# particionar agrega una columna llamada fold a un dataset
#   que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30),
#  agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30

def particionar(data: pl.DataFrame, division, agrupa="", campo="fold", start=1, seed=None):
    """
    Replica la función R particionar() usando Polars.
    
    Args:
        data: DataFrame de Polars
        division: Lista con las proporciones de cada fold (ej: [70, 30])
        agrupa: Nombre de la columna para estratificación (opcional)
        campo: Nombre de la columna resultante (default: "fold")
        start: Número inicial para los folds (default: 1)
        seed: Semilla para reproducibilidad (opcional)
    
    Returns:
        DataFrame con la columna de folds agregada
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Crear los bloques repetidos según la división
    bloques = np.concatenate([
        np.repeat(i, d) for i, d in enumerate(division, start=start)
    ])
    
    def asignar_fold(df):
        n = len(df)
        # Mezclar los bloques y repetir hasta cubrir todas las filas
        folds = np.resize(np.random.permutation(bloques), n)
        return df.with_columns(pl.Series(campo, folds))
    
    if agrupa and agrupa != "":
        # Estratificación por grupo
        result = (
            data.group_by(agrupa, maintain_order=True)
            .map_groups(asignar_fold)
        )
    else:
        # Sin estratificación
        n = data.height
        folds = np.resize(np.random.permutation(bloques), n)
        result = data.with_columns(pl.Series(campo, folds))
    
    return result

In [14]:
# DataFrame de ejemplo
# Crear datos de ejemplo
df = pl.DataFrame({
    "id": range(1, 21),
    "clase_ternaria": ["A"] * 10 + ["B"] * 10
})

# Partición 70/30 estratificada
df_particionado = particionar(
    data=df, 
    division=[70, 30], 
    agrupa="clase_ternaria", 
    seed=123
)

print("Resultado:")
print(df_particionado.sort(["clase_ternaria", "fold"]))
print("\nConteo por fold:")
print(df_particionado['fold'].value_counts())

Resultado:
shape: (20, 3)
┌─────┬────────────────┬──────┐
│ id  ┆ clase_ternaria ┆ fold │
│ --- ┆ ---            ┆ ---  │
│ i64 ┆ str            ┆ i64  │
╞═════╪════════════════╪══════╡
│ 1   ┆ A              ┆ 1    │
│ 4   ┆ A              ┆ 1    │
│ 5   ┆ A              ┆ 1    │
│ 6   ┆ A              ┆ 1    │
│ 7   ┆ A              ┆ 1    │
│ …   ┆ …              ┆ …    │
│ 11  ┆ B              ┆ 2    │
│ 12  ┆ B              ┆ 2    │
│ 17  ┆ B              ┆ 2    │
│ 18  ┆ B              ┆ 2    │
│ 19  ┆ B              ┆ 2    │
└─────┴────────────────┴──────┘

Conteo por fold:
shape: (2, 2)
┌──────┬───────┐
│ fold ┆ count │
│ ---  ┆ ---   │
│ i64  ┆ u32   │
╞══════╪═══════╡
│ 2    ┆ 8     │
│ 1    ┆ 12    │
└──────┴───────┘


In [15]:
def realidad_inicializar(pfuture: pl.DataFrame, pparam: dict) -> pl.DataFrame:
    """
    Inicializa el dataset de realidad para medir la ganancia.
    Replica la función R realidad_inicializar().
    
    Args:
        pfuture: DataFrame con los datos del futuro
        pparam: Diccionario con los parámetros (debe contener 'semilla_kaggle')
    
    Returns:
        DataFrame con las columnas necesarias y la partición aplicada
    """
    # Seleccionar solo las columnas necesarias (equivalente a pfuture[, list(numero_de_cliente, foto_mes, clase_ternaria)])
    drealidad = pfuture.select(["numero_de_cliente", "foto_mes", "clase_ternaria"])
    
    # Aplicar la partición usando la función particionar que creamos antes
    drealidad = particionar(
        data=drealidad,
        division=[3, 7],  # 30% y 70%
        agrupa="clase_ternaria",
        seed=pparam["semilla_kaggle"]
    )
    
    return drealidad

In [16]:
def test_particionar_y_realidad():
    """
    Función de prueba para verificar que ambas funciones trabajen correctamente
    """
    print("=== Probando función particionar ===")
    
    # Crear datos de ejemplo
    df_test = pl.DataFrame({
        "id": range(1, 21),
        "clase_ternaria": ["A"] * 10 + ["B"] * 10
    })
    
    # Test 1: Partición básica
    df_particionado = particionar(
        data=df_test, 
        division=[70, 30], 
        agrupa="clase_ternaria", 
        seed=123
    )
    
    print("Resultado partición 70/30:")
    print(df_particionado.sort(["clase_ternaria", "fold"]))
    print("\nConteo por fold:")
    print(df_particionado['fold'].value_counts())
    
    # Test 2: Verificar estratificación
    print("\nVerificación estratificación:")
    for clase in ["A", "B"]:
        subset = df_particionado.filter(pl.col("clase_ternaria") == clase)
        counts = subset['fold'].value_counts()
        print(f"Clase {clase}: {counts}")
    
    print("\n=== Probando función realidad_inicializar ===")
    
    # Crear datos simulando el futuro
    df_future = pl.DataFrame({
        "numero_de_cliente": range(1, 101),
        "foto_mes": [202104] * 100,  # Mes del futuro
        "clase_ternaria": ["CONTINUA"] * 80 + ["BAJA+1"] * 15 + ["BAJA+2"] * 5,
        "otra_columna": range(101, 201)  # Columna extra que no debe aparecer
    })
    
    # Parámetros de prueba
    param_test = {
        "semilla_kaggle": 314159
    }
    
    # Inicializar realidad
    drealidad = realidad_inicializar(df_future, param_test)
    
    print("Dataset de realidad inicializado:")
    print(f"Shape: {drealidad.shape}")
    print(f"Columnas: {drealidad.columns}")
    print("\nPrimeras filas:")
    print(drealidad.head(10))
    
    print("\nConteo por fold:")
    print(drealidad['fold'].value_counts())
    
    print("\nVerificación estratificación en realidad:")
    for clase in ["CONTINUA", "BAJA+1", "BAJA+2"]:
        subset = drealidad.filter(pl.col("clase_ternaria") == clase)
        if len(subset) > 0:
            counts = subset['fold'].value_counts()
            print(f"Clase {clase}: {counts}")
    
    return df_particionado, drealidad

# Ejecutar las pruebas
df_particionado, drealidad = test_particionar_y_realidad()

=== Probando función particionar ===
Resultado partición 70/30:
shape: (20, 3)
┌─────┬────────────────┬──────┐
│ id  ┆ clase_ternaria ┆ fold │
│ --- ┆ ---            ┆ ---  │
│ i64 ┆ str            ┆ i64  │
╞═════╪════════════════╪══════╡
│ 1   ┆ A              ┆ 1    │
│ 4   ┆ A              ┆ 1    │
│ 5   ┆ A              ┆ 1    │
│ 6   ┆ A              ┆ 1    │
│ 7   ┆ A              ┆ 1    │
│ …   ┆ …              ┆ …    │
│ 11  ┆ B              ┆ 2    │
│ 12  ┆ B              ┆ 2    │
│ 17  ┆ B              ┆ 2    │
│ 18  ┆ B              ┆ 2    │
│ 19  ┆ B              ┆ 2    │
└─────┴────────────────┴──────┘

Conteo por fold:
shape: (2, 2)
┌──────┬───────┐
│ fold ┆ count │
│ ---  ┆ ---   │
│ i64  ┆ u32   │
╞══════╪═══════╡
│ 2    ┆ 8     │
│ 1    ┆ 12    │
└──────┴───────┘

Verificación estratificación:
Clase A: shape: (2, 2)
┌──────┬───────┐
│ fold ┆ count │
│ ---  ┆ ---   │
│ i64  ┆ u32   │
╞══════╪═══════╡
│ 1    ┆ 7     │
│ 2    ┆ 3     │
└──────┴───────┘
Clase B: shape: (2,

In [17]:
# Evalúa la ganancia en los datos de la realidad usando polars

def realidad_evaluar(prealidad: pl.DataFrame, pprediccion: pl.DataFrame) -> dict:
    """
    Evalúa la ganancia en los datos de la realidad.
    Replica la función R realidad_evaluar().
    
    Args:
        prealidad: DataFrame con la realidad (debe tener columnas: numero_de_cliente, foto_mes, fold, clase_ternaria)
        pprediccion: DataFrame con las predicciones (debe tener columnas: numero_de_cliente, foto_mes, Predicted)
    
    Returns:
        dict: Diccionario con las ganancias 'public', 'private' y 'total'
    """
    # Join de la realidad con las predicciones (equivalente a prealidad[pprediccion, on=...])
    df_joined = prealidad.join(
        pprediccion.select(["numero_de_cliente", "foto_mes", "Predicted"]),
        on=["numero_de_cliente", "foto_mes"],
        how="left"
    )
    
    # Agrupar por fold, predicted, clase_ternaria y contar (equivalente a tbl <- prealidad[, list("qty"=.N), list(fold, predicted, clase_ternaria)])
    tbl = (
        df_joined
        .group_by(["fold", "Predicted", "clase_ternaria"])
        .agg(pl.len().alias("qty"))
    )
    
    # Calcular ganancia por registro
    tbl = tbl.with_columns(
        pl.when(pl.col("clase_ternaria") == "BAJA+2")
        .then(pl.col("qty") * 780000)
        .otherwise(pl.col("qty") * -20000)
        .alias("ganancia")
    )
    
    # Calcular métricas
    res = {}
    
    # Ganancia pública: fold==1 y predicted==1, dividido por 0.3
    public_ganancia = (
        tbl.filter((pl.col("fold") == 1) & (pl.col("Predicted") == 1))
        .select(pl.col("ganancia").sum())
        .item()
    )
    res["public"] = public_ganancia / 0.3 if public_ganancia is not None else 0
    
    # Ganancia privada: fold==2 y predicted==1, dividido por 0.7
    private_ganancia = (
        tbl.filter((pl.col("fold") == 2) & (pl.col("Predicted") == 1))
        .select(pl.col("ganancia").sum())
        .item()
    )
    res["private"] = private_ganancia / 0.7 if private_ganancia is not None else 0
    
    # Ganancia total: predicted==1
    total_ganancia = (
        tbl.filter(pl.col("Predicted") == 1)
        .select(pl.col("ganancia").sum())
        .item()
    )
    res["total"] = total_ganancia if total_ganancia is not None else 0
    
    return res

In [18]:
def test_realidad_evaluar():
    """
    Función de prueba para verificar que realidad_evaluar funcione correctamente
    """
    print("=== Probando función realidad_evaluar ===")
    
    # Crear datos de realidad simulados
    drealidad = pl.DataFrame({
        "numero_de_cliente": list(range(1, 101)),
        "foto_mes": [202104] * 100,
        "fold": [1] * 30 + [2] * 70,  # 30% público, 70% privado
        "clase_ternaria": ["CONTINUA"] * 80 + ["BAJA+1"] * 15 + ["BAJA+2"] * 5
    })
    
    # Crear predicciones simuladas (algunos clientes seleccionados)
    pprediccion = pl.DataFrame({
        "numero_de_cliente": [1, 5, 10, 15, 20, 25, 30, 35, 40, 45],  # 10 clientes seleccionados
        "foto_mes": [202104] * 10,
        "Predicted": [1] * 10  # Todos predichos como BAJA
    })
    
    print("Datos de realidad:")
    print(f"Total registros: {drealidad.height}")
    print("Distribución por fold:")
    print(drealidad['fold'].value_counts())
    print("Distribución por clase:")
    print(drealidad['clase_ternaria'].value_counts())
    
    print("\nPredicciones:")
    print(f"Clientes seleccionados: {pprediccion.height}")
    print(pprediccion)
    
    # Evaluar ganancia
    resultado = realidad_evaluar(drealidad, pprediccion)
    
    print("\n=== Resultados de ganancia ===")
    print(f"Ganancia Pública (fold=1): {resultado['public']:,.0f}")
    print(f"Ganancia Privada (fold=2): {resultado['private']:,.0f}")
    print(f"Ganancia Total: {resultado['total']:,.0f}")
    
    # Verificar manualmente algunos cálculos
    print("\n=== Verificación manual ===")
    
    # Join para ver qué clientes fueron seleccionados y sus clases
    df_joined = drealidad.join(
        pprediccion.select(["numero_de_cliente", "Predicted"]),
        on=["numero_de_cliente"],
        how="inner"
    )
    
    print("Clientes seleccionados con sus clases:")
    print(df_joined.select(["numero_de_cliente", "fold", "clase_ternaria"]))
    
    # Calcular ganancia manual
    ganancia_manual = 0
    for row in df_joined.iter_rows(named=True):
        if row["clase_ternaria"] == "BAJA+2":
            ganancia_manual += 780000
        else:
            ganancia_manual += -20000
    
    print(f"\nGanancia manual total: {ganancia_manual:,.0f}")
    print(f"Coincide con función: {abs(ganancia_manual - resultado['total']) < 1}")
    
    return resultado

# Ejecutar la prueba
resultado = test_realidad_evaluar()

=== Probando función realidad_evaluar ===
Datos de realidad:
Total registros: 100
Distribución por fold:
shape: (2, 2)
┌──────┬───────┐
│ fold ┆ count │
│ ---  ┆ ---   │
│ i64  ┆ u32   │
╞══════╪═══════╡
│ 2    ┆ 70    │
│ 1    ┆ 30    │
└──────┴───────┘
Distribución por clase:
shape: (3, 2)
┌────────────────┬───────┐
│ clase_ternaria ┆ count │
│ ---            ┆ ---   │
│ str            ┆ u32   │
╞════════════════╪═══════╡
│ BAJA+2         ┆ 5     │
│ CONTINUA       ┆ 80    │
│ BAJA+1         ┆ 15    │
└────────────────┴───────┘

Predicciones:
Clientes seleccionados: 10
shape: (10, 3)
┌───────────────────┬──────────┬───────────┐
│ numero_de_cliente ┆ foto_mes ┆ Predicted │
│ ---               ┆ ---      ┆ ---       │
│ i64               ┆ i64      ┆ i64       │
╞═══════════════════╪══════════╪═══════════╡
│ 1                 ┆ 202104   ┆ 1         │
│ 5                 ┆ 202104   ┆ 1         │
│ 10                ┆ 202104   ┆ 1         │
│ 15                ┆ 202104   ┆ 1         │
│ 

### Preprocesamiento

In [19]:
# carpeta de trabajo

# Definir la carpeta de trabajo y crearla si no existe
base_path = "exp"
experimento_folder = f"HT{PARAM['experimento']}"
experimento_path = os.path.join(base_path, experimento_folder)

# Crear la carpeta del experimento si no existe
os.makedirs(experimento_path, exist_ok=True)

In [20]:
# Lectura del dataset
dataset = pl.read_csv("datasets/competencia_01.csv.gz", infer_schema_length=100000)
dataset.head()

numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,tcuentas,ccuenta_corriente,mcuenta_corriente_adicional,mcuenta_corriente,ccaja_ahorro,mcaja_ahorro,mcaja_ahorro_adicional,mcaja_ahorro_dolares,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_personales,mprestamos_personales,cprestamos_prendarios,mprestamos_prendarios,cprestamos_hipotecarios,…,Master_mconsumospesos,Master_mconsumosdolares,Master_mlimitecompra,Master_madelantopesos,Master_madelantodolares,Master_fultimo_cierre,Master_mpagado,Master_mpagospesos,Master_mpagosdolares,Master_fechaalta,Master_mconsumototal,Master_cconsumos,Master_cadelantosefectivo,Master_mpagominimo,Visa_delinquency,Visa_status,Visa_mfinanciacion_limite,Visa_Fvencimiento,Visa_Finiciomora,Visa_msaldototal,Visa_msaldopesos,Visa_msaldodolares,Visa_mconsumospesos,Visa_mconsumosdolares,Visa_mlimitecompra,Visa_madelantopesos,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i64,i64,i64,f64,f64,i64,f64,f64,f64,i64,f64,i64,i64,f64,i64,i64,f64,i64,i64,f64,i64,f64,i64,f64,i64,…,f64,f64,f64,f64,f64,i64,f64,f64,f64,i64,f64,i64,i64,f64,i64,i64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,i64,f64,i64,i64,f64,str
249221323,202101,1,0,0,46,93,3070.24,15691.1,1278.11,890.96,636.86,8,1,1,0.0,-314.48,2,25761.53,0.0,10240.09,1,22890.77,1,5,35013.67,1,10,15929.92,1,13,27145.02,0,0.0,0,0.0,0,…,10853.35,58.6,70380.0,0.0,0.0,4,0.0,-10385.48,0.0,2814,10853.35,7,0,9290.16,0,0,63342.0,-849,null,43251.02,50733.44,0.0,6743.38,1.36,70380.0,0.0,0.0,4,0.0,-20833.47,0.0,2814,6743.38,6,0,9559.95,"""CONTINUA"""
249221323,202102,1,0,0,46,94,3385.85,19103.66,1230.47,1631.3,206.19,8,1,1,0.0,0.0,2,8116.43,0.0,10536.47,1,10629.59,1,1,2315.5,1,11,17285.39,1,22,25713.45,0,0.0,0,0.0,0,…,10910.41,56.8,70380.0,0.0,0.0,-3,42765.48,-42426.83,59.3,2842,10910.41,17,0,8539.44,0,0,63342.0,-821,null,19687.47,23093.4,0.0,7715.26,1.77,70380.0,0.0,0.0,-3,50733.44,-50733.44,0.0,2842,7715.26,8,0,7413.36,"""CONTINUA"""
249221323,202103,1,0,0,46,95,5341.18,24020.21,755.85,2768.95,1140.11,8,1,1,0.0,-3933.18,2,4015.77,0.0,10789.84,1,4650.31,1,0,0.0,1,10,18976.09,1,16,45821.57,0,0.0,0,0.0,0,…,30992.11,39.83,70380.0,0.0,0.0,1,0.0,-42534.65,57.48,2873,30992.11,11,0,0.0,0,0,63342.0,-790,null,-2453.06,-2877.43,0.0,10703.0,1.53,70380.0,0.0,0.0,1,0.0,-50733.44,0.0,2873,10703.0,7,0,0.0,"""CONTINUA"""
249221323,202104,1,0,0,46,96,5356.62,29189.81,1470.2,2627.7,685.53,8,1,1,0.0,-1960.57,2,2136.11,0.0,10974.0,1,9358.46,1,0,0.0,1,10,19201.77,1,18,17373.08,0,0.0,0,0.0,0,…,10360.48,49.19,70380.0,0.0,0.0,2,0.0,-42579.46,40.31,2903,10360.48,14,0,4821.03,0,0,63342.0,-760,null,14545.66,17062.05,0.0,8094.73,1.51,70380.0,0.0,0.0,2,0.0,0.0,0.0,2903,8094.73,6,0,0.0,"""CONTINUA"""
249221323,202105,1,0,0,46,97,2489.25,31505.78,1008.8,952.16,309.95,8,1,1,0.0,-263.5,2,6204.56,0.0,11106.55,1,13240.63,1,1,2216.97,1,9,19278.3,1,14,16708.69,0,0.0,0,0.0,0,…,7293.92,43.34,70380.0,0.0,0.0,5,0.0,-21115.45,49.78,2934,7293.92,10,0,4586.43,0,0,63342.0,-729,null,17064.74,20016.94,0.0,9051.98,1.48,70380.0,0.0,0.0,5,0.0,-17062.05,0.0,2934,9051.98,6,0,2111.4,null


In [21]:
#dataset = dataset.group_by("clase_ternaria", maintain_order=True).map_groups(lambda group: group.sample(fraction=0.1, seed=42))

In [22]:
dataset.shape

(978439, 155)

In [23]:
### Lags y deltas

# ordenar por cliente y mes para que los shifts sean lags reales
dataset = dataset.sort(["numero_de_cliente", "foto_mes"])

# columnas numéricas (excluyendo id/tiempo/target)
excluir = {"numero_de_cliente", "foto_mes", "clase_ternaria"}
num_cols = [c for c, dt in zip(dataset.columns, dataset.dtypes)
            if dt in pl.NUMERIC_DTYPES and c not in excluir]

# lags 1 y 2 + deltas 1 y 2 por cliente
exprs = []
for c in num_cols:
    exprs += [
        pl.col(c).shift(1).over("numero_de_cliente").alias(f"{c}_lag1"),
        pl.col(c).shift(2).over("numero_de_cliente").alias(f"{c}_lag2"),
        (pl.col(c) - pl.col(c).shift(1).over("numero_de_cliente")).alias(f"{c}_delta1"),
        (pl.col(c) - pl.col(c).shift(2).over("numero_de_cliente")).alias(f"{c}_delta2"),
    ]

dataset = dataset.with_columns(exprs)

# vista rápida
print(dataset.select(["numero_de_cliente","foto_mes"] + [f"{num_cols[0]}_lag1", f"{num_cols[0]}_delta1"]).head())


/tmp/ipykernel_1878865/1420011140.py:9: DeprecationWarning: `NUMERIC_DTYPES` was deprecated in version 1.0.0. Define your own data type groups or use the `polars.selectors` module for selecting columns of a certain data type.
  if dt in pl.NUMERIC_DTYPES and c not in excluir]


shape: (5, 4)
┌───────────────────┬──────────┬─────────────────────┬───────────────────────┐
│ numero_de_cliente ┆ foto_mes ┆ active_quarter_lag1 ┆ active_quarter_delta1 │
│ ---               ┆ ---      ┆ ---                 ┆ ---                   │
│ i64               ┆ i64      ┆ i64                 ┆ i64                   │
╞═══════════════════╪══════════╪═════════════════════╪═══════════════════════╡
│ 249221323         ┆ 202101   ┆ null                ┆ null                  │
│ 249221323         ┆ 202102   ┆ 1                   ┆ 0                     │
│ 249221323         ┆ 202103   ┆ 1                   ┆ 0                     │
│ 249221323         ┆ 202104   ┆ 1                   ┆ 0                     │
│ 249221323         ┆ 202105   ┆ 1                   ┆ 0                     │
└───────────────────┴──────────┴─────────────────────┴───────────────────────┘


In [24]:
# Intentamos identificar quien recibe aguinaldo
# Creo un criterio de identificación de aguinaldo, y la modifico con un factor de proporcion

for var in num_cols:
    mediana_hist = dataset.filter(pl.col("foto_mes") < 202106)[var].median()
    mediana_junio = dataset.filter(pl.col("foto_mes") == 202106)[var].median()

    # Evitar división por cero
    if mediana_junio == 0 or mediana_junio is None:
        factor = 1.0
    else:
        factor = mediana_hist / mediana_junio

    # Corrige solo en junio 202106 para la columna correspondiente
    dataset = dataset.with_columns(
        pl.when(pl.col("foto_mes") == 202106)
        .then(pl.col(var) * factor)
        .otherwise(pl.col(var))
        .alias(var)
    )

In [25]:
print(dataset.select(["numero_de_cliente","foto_mes", "mpayroll", "mpayroll_lag1", "mpayroll_delta1"]).head())


shape: (5, 5)
┌───────────────────┬──────────┬───────────┬───────────────┬─────────────────┐
│ numero_de_cliente ┆ foto_mes ┆ mpayroll  ┆ mpayroll_lag1 ┆ mpayroll_delta1 │
│ ---               ┆ ---      ┆ ---       ┆ ---           ┆ ---             │
│ i64               ┆ i64      ┆ f64       ┆ f64           ┆ f64             │
╞═══════════════════╪══════════╪═══════════╪═══════════════╪═════════════════╡
│ 249221323         ┆ 202101   ┆ 97524.39  ┆ null          ┆ null            │
│ 249221323         ┆ 202102   ┆ 97055.19  ┆ 97524.39      ┆ -469.2          │
│ 249221323         ┆ 202103   ┆ 97289.79  ┆ 97055.19      ┆ 234.6           │
│ 249221323         ┆ 202104   ┆ 97641.69  ┆ 97289.79      ┆ 351.9           │
│ 249221323         ┆ 202105   ┆ 113852.55 ┆ 97641.69      ┆ 16210.86        │
└───────────────────┴──────────┴───────────┴───────────────┴─────────────────┘


In [26]:
# los campos que se van a utilizar

excluir = {"clase_ternaria","clase01","azar","training","periodo0","periodo1","periodo2"}
campos_buenos = [c for c in dataset.columns if c not in excluir]
print(campos_buenos)
print(len(campos_buenos))

['numero_de_cliente', 'foto_mes', 'active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'cproductos', 'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 'ccaja_ahorro', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'cdescubierto_preacordado', 'mcuentas_saldo', 'ctarjeta_debito', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master', 'ctarjeta_master_transacciones', 'mtarjeta_master_consumo', 'cprestamos_personales', 'mprestamos_personales', 'cprestamos_prendarios', 'mprestamos_prendarios', 'cprestamos_hipotecarios', 'mprestamos_hipotecarios', 'cplazo_fijo', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'cinversion1', 'minversion1_pesos', 'minversion1_dolares', 'cinversion2', 'minversion2', 'cseguro_vida', 'cseguro_auto', 'csegur

## Produccion

### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

In [27]:
base_exp_dir = "exp"
experimento = f"exp{PARAM['experimento']}"
experimento_path = os.path.join(base_exp_dir, experimento)

os.makedirs(experimento_path, exist_ok=True)

#### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la  optimización bayesiana

In [28]:
# clase01
dataset = dataset.with_columns(
    (pl.col("clase_ternaria").is_in(["BAJA+2", "BAJA+1"]).cast(pl.Int8)).alias("clase01")
)

In [29]:
# --- Filtrar por los meses de entrenamiento final ---
dataset_train = dataset.filter(pl.col("foto_mes").is_in(PARAM["train_final"]))

resumen = (
    dataset_train
    .group_by("clase_ternaria")
    .len()
    .rename({"len": "N"})
    .sort("clase_ternaria")
)

print(resumen)

shape: (3, 2)
┌────────────────┬────────┐
│ clase_ternaria ┆ N      │
│ ---            ┆ ---    │
│ str            ┆ u32    │
╞════════════════╪════════╡
│ BAJA+1         ┆ 2492   │
│ BAJA+2         ┆ 2808   │
│ CONTINUA       ┆ 481491 │
└────────────────┴────────┘


In [30]:
# dejo los datos en el formato que necesita LightGBM
# Features y label sobre todo el dataset_train (sin máscara)
X_final = (
    dataset_train
    .select(campos_buenos)
    # opcional: emular data.matrix y forzar numérico
    # .with_columns(pl.all().cast(pl.Float32))
    .to_numpy()
)
y_final = dataset_train["clase01"].to_numpy()

dtrain_final = lgb.Dataset(
    data=X_final,
    label=y_final,
    free_raw_data=False
)

print("nrow_final:", X_final.shape[0])
print("ncol_final:", X_final.shape[1])

nrow_final: 486791
ncol_final: 762


#### Final Training Hyperparameters

In [32]:
# Fusionar parámetros fijos con los óptimos encontrados
# Cargar best parameters
with open("PARAM_4942.yml", "r") as file:
    param_data = yaml.safe_load(file)

best_params = param_data["out"]["lgbm"]["mejores_hiperparametros"]

param_final = {**PARAM["lgbm"]["param_fijos"], **best_params}

# Mostrar el resultado final
print("Parámetros finales de LightGBM:")
for k, v in param_final.items():
    print(f" {k}: {v}")

Parámetros finales de LightGBM:
 boosting: gbdt
 objective: binary
 metric: auc
 first_metric_only: False
 boost_from_average: True
 feature_pre_filter: False
 force_row_wise: True
 verbosity: -100
 seed: 100129
 min_gain_to_split: 0
 min_sum_hessian_in_leaf: 0.001
 max_bin: 31
 pos_bagging_fraction: 1.0
 neg_bagging_fraction: 1.0
 is_unbalance: False
 scale_pos_weight: 1.0
 max_drop: 50
 skip_drop: 0.5
 extra_trees: False
 num_iterations: 1200
 learning_rate: 0.04481089925392504
 feature_fraction: 0.5170503571975581
 min_data_in_leaf: 296
 num_leaves: 2341
 max_depth: 14
 lambda_l1: 0.4514108759697508
 lambda_l2: 9.426450679561881
 bagging_fraction: 0.886052537529134


#### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria y mucho menos cross validation.

In [33]:
# Copiar los parámetros finales
param_normalizado = param_final.copy()

# Ajustar min_data_in_leaf según el undersampling usado en la BO
param_normalizado["min_data_in_leaf"] = round(
    param_final["min_data_in_leaf"] / PARAM["trainingstrategy"]["undersampling"]
)

print(" Parámetros normalizados:")
print(f"min_data_in_leaf ajustado: {param_normalizado['min_data_in_leaf']}")

 Parámetros normalizados:
min_data_in_leaf ajustado: 296


In [34]:
# Entrenar el modelo final de LightGBM
modelo_final = lgb.train(
    params=param_normalizado,
    train_set=dtrain_final
)

In [35]:
gain = modelo_final.feature_importance(importance_type="gain")
split = modelo_final.feature_importance(importance_type="split")
feat = modelo_final.feature_name()

tb_importancia = pl.DataFrame({
    "Feature": feat,
    "Gain": gain,
    "Split": split
})

tb_importancia = tb_importancia.with_columns(
    pl.Series("Feature", campos_buenos)
)

archivo_importancia = os.path.join(experimento_path, "impo.txt")
tb_importancia.write_csv(archivo_importancia, separator="\t")


In [36]:
# Definir ruta completa dentro del experimento
modelo_path = os.path.join(experimento_path, "modelo.txt")

# Guardar el modelo final en formato texto (igual que R)
modelo_final.save_model(modelo_path)
print(f"Modelo final guardado en: {modelo_path}")

Modelo final guardado en: exp/exp49422/modelo.txt


### Scoring

Aplico el modelo final a los datos del futuro

In [37]:
# aplico el modelo a los datos sin clase
dfuture = dataset.filter(pl.col("foto_mes").is_in(PARAM["future"]))

# aplicar el modelo a los datos nuevos
prediccion = modelo_final.predict(
    dfuture.select(campos_buenos).to_numpy()
)

In [38]:
# inicilizo el dataset drealidad
drealidad = realidad_inicializar(dfuture, PARAM)

In [39]:
drealidad.shape

(163418, 4)

#### Tabla Prediccion

In [40]:
# tabla de prediccion
tb_prediccion = (
    dfuture
    .select(["numero_de_cliente", "foto_mes"])
    .with_columns(pl.Series("prob", prediccion))
)

Kaggle Competition Submit

In [41]:
print(PARAM["cortes"])

[6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500, 10000, 10500, 11000, 11500, 12000, 12500, 13000, 13500, 14000, 14500, 15000, 15500, 16000, 16500, 17000, 17500, 18000, 18500, 19000]


In [42]:
# ordenar por probabilidad descendente
tb_prediccion_sorted = tb_prediccion.sort("prob", descending=True)

for envios in PARAM["cortes"]:
    pred = (
        tb_prediccion_sorted
        .with_row_index("rn")  # <- reemplaza with_row_count
        .with_columns(
            pl.when(pl.col("rn") < envios).then(1).otherwise(0).alias("Predicted")
        )
        .drop("rn")
    )
    res = realidad_evaluar(drealidad, pred)
    print(f"Envios={envios}\t TOTAL={res['total']}  Public={res['public']} Private={res['private']}")


Envios=6000	 TOTAL=343200000  Public=375000000.0 Private=329571428.5714286
Envios=6500	 TOTAL=348400000  Public=384600000.0 Private=332885714.2857143
Envios=7000	 TOTAL=352000000  Public=382400000.0 Private=338971428.5714286
Envios=7500	 TOTAL=358000000  Public=388066666.6666667 Private=345114285.71428573
Envios=8000	 TOTAL=357600000  Public=384200000.0 Private=346200000.0
Envios=8500	 TOTAL=357200000  Public=382600000.0 Private=346314285.71428573
Envios=9000	 TOTAL=354400000  Public=383733333.3333334 Private=341828571.42857146
Envios=9500	 TOTAL=355600000  Public=379066666.6666667 Private=345542857.1428572
Envios=10000	 TOTAL=354400000  Public=375400000.0 Private=345400000.0
Envios=10500	 TOTAL=356400000  Public=371133333.3333334 Private=350085714.2857143
Envios=11000	 TOTAL=352000000  Public=365866666.6666667 Private=346057142.85714287
Envios=11500	 TOTAL=350800000  Public=363333333.3333334 Private=345428571.42857146
Envios=12000	 TOTAL=348000000  Public=366733333.3333334 Private=339

In [43]:
param_path = os.path.join(experimento_path, "PARAM.yml")

# Guardar PARAM en YAML
with open(param_path, "w", encoding="utf-8") as f:
    yaml.safe_dump(PARAM, f, sort_keys=False, allow_unicode=True)

In [44]:
print(datetime.now().strftime("%a %b %d %X %Y"))

Sun Oct 12 15:46:37 2025


## Kaggle Submission

In [45]:
# datos sin clase (kaggle)
dfuture = dataset.filter(pl.col("foto_mes").is_in(PARAM["kaggle"]))

# aplicar el modelo a los datos nuevos
X_future = dfuture.select(campos_buenos).to_numpy()
prediccion = modelo_final.predict(X_future)

# dimensiones y tabla de foto_mes
print(dfuture.shape)
print(
    dfuture.group_by("foto_mes")
           .len()
           .rename({"len": "N"})
           .sort("foto_mes")
)

(164313, 764)
shape: (1, 2)
┌──────────┬────────┐
│ foto_mes ┆ N      │
│ ---      ┆ ---    │
│ i64      ┆ u32    │
╞══════════╪════════╡
│ 202106   ┆ 164313 │
└──────────┴────────┘


In [46]:
# inicilizo el dataset  drealidad
drealidad = realidad_inicializar(dfuture, PARAM)
drealidad.shape

(164313, 4)

In [47]:
# tabla de predicción
tb_prediccion = (
    dfuture
    .select(["numero_de_cliente", "foto_mes"])
    .with_columns(pl.Series("prob", prediccion))
)

# grabo las probabilidades del modelo
archivo_prediccion = os.path.join(experimento_path, "prediccion.txt")
tb_prediccion.write_csv(archivo_prediccion, separator="\t")

In [48]:
# ordenar por probabilidad descendente
tb_prediccion_sorted = tb_prediccion.sort("prob", descending=True)

# crear carpeta
kaggle_path = os.path.join(experimento_path, "kaggle")
os.makedirs(kaggle_path, exist_ok=True)

for envios in PARAM["cortes"]:
    # Predicted = 1 para los primeros 'envios', 0 para el resto
    pred = (
        tb_prediccion_sorted
        .with_row_index("rn")
        .with_columns(
            pl.when(pl.col("rn") < envios).then(1).otherwise(0).alias("Predicted")
        )
        .select(["numero_de_cliente", "Predicted"])
    )

    archivo_kaggle = os.path.join(kaggle_path, f"KA{PARAM['experimento']}_{envios}.csv")
    pred.write_csv(archivo_kaggle)
